In [5]:
from urllib import request
# Define the remote file to retrieve
remote_url = 'https://today.yougov.com/_pubapis/v5/us/trackers/president-obama-job-approval-rating/download/'
# Define the local filename to save data
local_file = 'obama-job-approval-ratings.xlsx'
# Download remote and save locally
request.urlretrieve(remote_url, local_file)

('obama-job-approval-ratings.xlsx', <http.client.HTTPMessage at 0x22d150cc040>)

In [1]:
!pip list

Package                           Version
--------------------------------- ---------------------
absl-py                           1.0.0
alabaster                         0.7.12
anyio                             3.5.0
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
astroid                           2.9.3
astropy                           5.0.1
asttokens                         2.0.5
astunparse                        1.6.3
async-generator                   1.10
attrs                             21.4.0
Babel                             2.9.1
backcall                          0.2.0
beautifulsoup4                    4.10.0
black                             22.1.0
bleach                            4.1.0
cachetools                        5.0.0
certifi                           2021.5.30
cffi                              1.15.0
chardet                           4.0.0
charset-normalizer                2.0.12
click                             8.0.3
colorama   

In [2]:
import pandas as pd
df = pd.read_excel('obama-job-approval-ratings.xlsx')
df.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.